## PCA 차원축소

라이브러리 준비

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

In [4]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
import awswrangler as wr

In [7]:
import os
import pickle
import sagemaker
import boto3
from dotenv import load_dotenv
load_dotenv();

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

SageMaker 세션 및 역할 설정

In [6]:
boto3_session = boto3.Session(profile_name='awstutor')
sagemaker_session = sagemaker.Session(boto_session=boto3_session)
role = os.environ.get('SAGEMAKER_EXECUTION_ROLE_ARN')

데이터셋 다운로드

In [36]:
from ucimlrepo import fetch_ucirepo 

# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 

데이터 확인

In [9]:
adult.metadata

{'uci_id': 2,
 'name': 'Adult',
 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult',
 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv',
 'abstract': 'Predict whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ',
 'area': 'Social Science',
 'tasks': ['Classification'],
 'characteristics': ['Multivariate'],
 'num_instances': 48842,
 'num_features': 14,
 'feature_types': ['Categorical', 'Integer'],
 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'],
 'target_col': ['income'],
 'index_col': None,
 'has_missing_values': 'yes',
 'missing_values_symbol': 'NaN',
 'year_of_dataset_creation': 1996,
 'last_updated': 'Mon Aug 07 2023',
 'dataset_doi': '10.24432/C5XW20',
 'creators': ['Barry Becker', 'Ronny Kohavi'],
 'intro_paper': None,
 'additional_info': {'summary': 'Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the follow

In [10]:
adult.variables

,name,role,type,demographic,description,units,missing_values
0,age,Feature,Integer,Age,N/A,None,no
1,workclass,Feature,Categorical,Income,"Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.",None,yes
2,fnlwgt,Feature,Integer,None,None,None,no
3,education,Feature,Categorical,Education Level,"Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.",None,no
4,education-num,Feature,Integer,Education Level,None,None,no
5,marital-status,Feature,Categorical,Other,"Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.",None,no
6,occupation,Feature,Categorical,Other,"Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.",None,yes
7,relationship,Feature,Categorical,Other,"Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.",None,no
8,race,Feature,Categorical,Race,"White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.",None,no
9,sex,Feature,Binary,Sex,"Female, Male.",None,no


In [21]:
y.value_counts()

income
<=50K     24720
<=50K.    12435
>50K       7841
>50K.      3846
Name: count, dtype: int64

In [12]:
X.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,48842.000000,4.884200e+04,48842.000000,48842.000000,48842.000000,48842.000000
mean,38.643585,1.896641e+05,10.078089,1079.067626,87.502314,40.422382
std,13.710510,1.056040e+05,2.570973,7452.019058,403.004552,12.391444
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.175505e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.781445e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.376420e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.490400e+06,16.000000,99999.000000,4356.000000,99.000000


In [13]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       47879 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      47876 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48568 non-null  object
dtypes: int64(6), object(8)
memory usage: 5.2+ MB


In [22]:
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


S3 경로 설정

In [39]:
bucket_name = 'dante-sagemaker'
project_name = 'adult-income-classification'

In [77]:
input_path = f's3://{bucket_name}/{project_name}/input'
output_path = f's3://{bucket_name}/{project_name}/output'
model_path = f's3://{bucket_name}/{project_name}/model'
asset_path = f's3://{bucket_name}/{project_name}/asset'
script_path = f's3://{bucket_name}/{project_name}/script'
checkpoint_path = f's3://{bucket_name}/{project_name}/checkpoints'
pca_path = f's3://{bucket_name}/{project_name}/pca'

train_path = f'{input_path}/train/train.recordio'
val_path = f'{input_path}/val/val.recordio'
test_path = f'{input_path}/test/test.csv'

print('train_path:', train_path)
print('val_path:', val_path)
print('test_path:', test_path)
print('model_path:', model_path)
print('asset_path:', asset_path)
print('checkpoint_path:', checkpoint_path)
print('script_path:', script_path)
print('pca_path:', pca_path)

train_path: s3://dante-sagemaker/adult-income-classification/input/train/train.recordio
val_path: s3://dante-sagemaker/adult-income-classification/input/val/val.recordio
test_path: s3://dante-sagemaker/adult-income-classification/input/test/test.csv
model_path: s3://dante-sagemaker/adult-income-classification/model
asset_path: s3://dante-sagemaker/adult-income-classification/asset
checkpoint_path: s3://dante-sagemaker/adult-income-classification/checkpoints
script_path: s3://dante-sagemaker/adult-income-classification/script
pca_path: s3://dante-sagemaker/adult-income-classification/pca


테스트 데이터 분리 및 S3 업로드

In [37]:
X_tmp, X_test, y_tmp, y_test = train_test_split(X, y, test_size = 0.1, random_state = 2024)

In [42]:
test_df = pd.concat([y_test, X_test], axis=1)
wr.s3.to_csv(test_df, test_path, index=False, boto3_session=boto3_session)

{'paths': ['s3://dante-sagemaker/adult-income-classification/input/test/test.csv'],
 'partitions_values': {}}

데이터 전처리

In [43]:
# '?' 값을 NaN으로 대체
X_tmp[X_tmp == '?'] = np.nan

In [44]:
# 범주형(object) 데이터 열 이름 확인
X_tmp.select_dtypes(include=['object']).columns

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object')

In [46]:
# workclass, occupation, native-country 열은 결측치가 존재하기 때문에 이를 최빈값을 보간합니다.
X_tmp['workclass'].fillna(X_tmp['workclass'].mode()[0], inplace=True)
X_tmp['occupation'].fillna(X_tmp['occupation'].mode()[0], inplace=True)
X_tmp['native-country'].fillna(X_tmp['native-country'].mode()[0], inplace=True)
X_tmp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43957 entries, 25825 to 7816
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             43957 non-null  int64 
 1   workclass       43957 non-null  object
 2   fnlwgt          43957 non-null  int64 
 3   education       43957 non-null  object
 4   education-num   43957 non-null  int64 
 5   marital-status  43957 non-null  object
 6   occupation      43957 non-null  object
 7   relationship    43957 non-null  object
 8   race            43957 non-null  object
 9   sex             43957 non-null  object
 10  capital-gain    43957 non-null  int64 
 11  capital-loss    43957 non-null  int64 
 12  hours-per-week  43957 non-null  int64 
 13  native-country  43957 non-null  object
dtypes: int64(6), object(8)
memory usage: 5.0+ MB


In [47]:
# 타겟 변수 'y'를 이진 분류를 위해 숫자로 변환
# '<=50K'와 '<=50K.'는 0으로, '>50K'와 '>50K.'는 1로 변환
y_tmp = y_tmp.replace({
    '<=50K': 0,
    '<=50K.': 0,
    '>50K': 1,
    '>50K.': 1
})
y_tmp.value_counts()

income
0         33456
1         10501
Name: count, dtype: int64

In [48]:
X_train, X_val, y_train, y_val = train_test_split(X_tmp, y_tmp, test_size = 0.3, random_state = 2024)

In [49]:
# 범주형 변수를 인코딩하기 위한 딕셔너리 생성
feature_encoders = {}

# 범주형(object) 데이터 열 선택
categorical_cols = X.select_dtypes(include=['object']).columns

# 각 범주형 열에 대해 레이블 인코딩 수행
for col in categorical_cols:
    # LabelEncoder 객체 생성
    encoder = LabelEncoder()
    
    # 훈련 데이터에 대해 인코더를 학습하고 변환
    X_train[col] = encoder.fit_transform(X_train[col])
    
    # 인코더를 딕셔너리에 저장
    feature_encoders[col] = encoder
    
    # 검증 데이터와 테스트 데이터에 대해 학습된 인코더로 변환
    X_val[col] = encoder.transform(X_val[col])

In [50]:
# StandardScaler를 사용하여 특성 스케일링 수행
scaler = StandardScaler()

# 훈련 데이터에 대해 스케일러를 학습하고 변환
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)

# 검증 데이터와 테스트 데이터에 대해 학습된 스케일러로 변환
X_val = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns)

In [51]:
# 숫자형 데이터로 모두 변환되었는지 확인
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30769 entries, 0 to 30768
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             30769 non-null  float64
 1   workclass       30769 non-null  float64
 2   fnlwgt          30769 non-null  float64
 3   education       30769 non-null  float64
 4   education-num   30769 non-null  float64
 5   marital-status  30769 non-null  float64
 6   occupation      30769 non-null  float64
 7   relationship    30769 non-null  float64
 8   race            30769 non-null  float64
 9   sex             30769 non-null  float64
 10  capital-gain    30769 non-null  float64
 11  capital-loss    30769 non-null  float64
 12  hours-per-week  30769 non-null  float64
 13  native-country  30769 non-null  float64
dtypes: float64(14)
memory usage: 3.3 MB


In [53]:
X_train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,-1.360951,-0.093884,-0.150088,1.214871,-0.027163,0.912707,-0.290514,0.971772,0.391761,0.704144,-0.145479,-0.215481,-1.649635,0.260168
1,1.044307,-0.093884,-1.514669,0.184108,-0.415719,-0.411967,-1.046313,-0.901651,0.391761,0.704144,-0.145479,-0.215481,-0.033765,0.260168
2,2.064719,2.600820,0.678506,-1.362036,-2.358501,-0.411967,1.724950,-0.901651,0.391761,0.704144,-0.145479,-0.215481,-0.437733,0.260168
3,-1.433838,-0.093884,0.288244,1.214871,-0.027163,0.912707,-0.542447,0.971772,0.391761,0.704144,-0.145479,-0.215481,0.774170,0.260168
4,-1.288065,-0.093884,-0.676139,1.214871,-0.027163,0.912707,1.221084,0.971772,-1.981795,-1.420164,-0.145479,-0.215481,-1.649635,0.260168


In [52]:
# 추후 테스트를 위해 인코더, 스케일러를 파일로 저장
os.makedirs('dataset', exist_ok=True)
with open('dataset/adult_encoders.pkl', 'wb') as f:
    pickle.dump(feature_encoders, f)
with open('dataset/adult_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [ ]:
# 인코더 업로드
encoders_s3_path = os.path.join(asset_path, 'adult_encoders.pkl')
wr.s3.upload('dataset/adult_encoders.pkl', encoders_s3_path, boto3_session=boto3_session)
# 스케일러 업로드
scaler_s3_path = os.path.join(asset_path, 'adult_scaler.pkl')
wr.s3.upload('dataset/adult_scaler.pkl', scaler_s3_path, boto3_session=boto3_session)

### 로컬 모드
---

차원 축소

In [54]:
pca = PCA(n_components=0.9)
X_train_pca = pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)

In [55]:
pca.explained_variance_ratio_

array([0.14650695, 0.10236428, 0.08092191, 0.07883121, 0.07453303,
       0.07323439, 0.07035841, 0.06760339, 0.06498243, 0.06107142,
       0.06064704, 0.04896272])

In [56]:
print(f"차원축소 : {X_train.shape} -> {X_train_pca.shape}")
print(f"차원축소 : {X_val.shape} -> {X_val_pca.shape}")

차원축소 : (30769, 14) -> (30769, 12)
차원축소 : (13188, 14) -> (13188, 12)


모델 훈련

In [57]:
# XGBoost 모델 생성 및 학습
xgb_model = XGBClassifier(random_state=2024)
xgb_model.fit(X_train_pca, y_train)

# 검증 세트에 대한 예측
y_val_pred = xgb_model.predict(X_val_pca)

# 모델 성능 평가
accuracy = accuracy_score(y_val, y_val_pred)
print(f"XGBoost 모델의 정확도: {accuracy:.4f}")

XGBoost 모델의 정확도: 0.8396


시각화

In [61]:
# plotly를 사용하여 결과 시각화
import plotly.graph_objects as go
import numpy as np

# 실제 값과 예측 값 비교
y_val_flat = y_val.values.flatten()  # 1차원으로 변환
y_val_pred_flat = y_val_pred.flatten()  # 1차원으로 변환
confusion_matrix = pd.crosstab(y_val_flat, y_val_pred_flat, rownames=['실제'], colnames=['예측'])

# 히트맵 생성
fig = go.Figure(data=go.Heatmap(
                z=confusion_matrix.values,
                x=confusion_matrix.columns,
                y=confusion_matrix.index,
                colorscale='Viridis'))

# 레이아웃 설정
fig.update_layout(
    title='혼동 행렬',
    xaxis_title='예측 값',
    yaxis_title='실제 값'
)

# 각 셀에 텍스트 추가
annotations = []
for i, row in enumerate(confusion_matrix.values):
    for j, value in enumerate(row):
        annotations.append(
            dict(
                x=confusion_matrix.columns[j],
                y=confusion_matrix.index[i],
                text=str(value),
                showarrow=False,
                font=dict(color='white' if value < confusion_matrix.values.max() / 2 else 'black')
            )
        )
fig.update_layout(annotations=annotations)

# 그래프 표시
fig.show()

In [62]:
# 특성 중요도 시각화
feature_importance = xgb_model.feature_importances_
feature_names = [f'PC{i+1}' for i in range(len(feature_importance))]

# 특성 중요도를 내림차순으로 정렬
sorted_idx = np.argsort(feature_importance)
sorted_feature_names = [feature_names[i] for i in sorted_idx]
sorted_feature_importance = feature_importance[sorted_idx]

# 막대 그래프 생성
fig = go.Figure(go.Bar(
    x=sorted_feature_importance,
    y=sorted_feature_names,
    orientation='h'
))

# 레이아웃 설정
fig.update_layout(
    title='XGBoost 모델의 특성 중요도',
    xaxis_title='중요도',
    yaxis_title='주성분',
    height=600,
    width=800
)

# 그래프 표시
fig.show()


### SageMaker 클라우드 모드
---

데이터 업로드

In [ ]:
# import io
# import numpy as np
# import sagemaker.amazon.common as smac
# import boto3

# def numpy_to_recordio_protobuf(array):
#     buffer = io.BytesIO()
#     smac.write_numpy_to_dense_tensor(buffer, array)
#     return buffer.getvalue()

# # S3 클라이언트 생성
# s3_client = boto3.client('s3')

# # 버킷 이름과 파일 키(경로) 설정
# bucket_name = 'your-bucket-name'
# train_key = 'path/to/train_data.recordio'
# val_key = 'path/to/val_data.recordio'

# # 데이터를 RecordIO-protobuf 형식으로 변환
# train_data_recordio = numpy_to_recordio_protobuf(X_train.values)
# val_data_recordio = numpy_to_recordio_protobuf(X_val.values)

# # S3에 업로드
# s3_client.put_object(Body=train_data_recordio, Bucket=bucket_name, Key=train_key)
# s3_client.put_object(Body=val_data_recordio, Bucket=bucket_name, Key=val_key)

In [78]:
from sagemaker.serializers import RecordSerializer
serializer = RecordSerializer()
wr.s3.upload(serializer.serialize(X_train.values), train_path, boto3_session=boto3_session)
wr.s3.upload(serializer.serialize(X_val.values), val_path, boto3_session=boto3_session)

PCA 빌트인 모델 및 하이퍼파라미터 설정

In [69]:
container = sagemaker.image_uris.retrieve('pca', sagemaker_session.boto_region_name)
use_spot_instances = True
max_run = 60 * 60 # 1시간
max_wait = 60 * 60 # 1시간
training_instance_type = "ml.m5.xlarge"
content_type = 'text/csv'

In [85]:
from sagemaker.utils import name_from_base

pca_estimator =  sagemaker.estimator.Estimator(
    container,
    role, 
    sagemaker_session=sagemaker_session,
    instance_count=1, 
    instance_type=training_instance_type,
    output_path=output_path,
    base_job_name = name_from_base(project_name),
    use_spot_instances=use_spot_instances,
    max_run=max_run,
    max_wait=max_wait,
)

In [86]:
# https://docs.aws.amazon.com/ko_kr/sagemaker/latest/dg/PCA-reference.html
pca_estimator.set_hyperparameters(
    feature_dim=14,
    num_components=12,
    subtract_mean=False,
    algorithm_mode='regular',
    mini_batch_size=200
)

In [87]:
pca_estimator.fit({'train': train_path})

INFO:sagemaker:Creating training-job with name: adult-income-classification-2024-08-01--2024-08-01-02-28-15-411


2024-08-01 02:28:33 Starting - Starting the training job...
2024-08-01 02:28:48 Starting - Preparing the instances for training...
2024-08-01 02:29:28 Downloading - Downloading the training image........Docker entrypoint called with argument(s): train
Running default environment configuration script
/opt/amazon/lib/python3.8/site-packages/mxnet/model.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if num_device is 1 and 'dist' not in kvstore:
[08/01/2024 02:30:48 INFO 140444590286656] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-conf.json: {'algorithm_mode': 'regular', 'subtract_mean': 'true', 'extra_components': '-1', 'force_dense': 'true', 'epochs': 1, '_log_level': 'info', '_kvstore': 'dist_sync', '_num_kv_servers': 'auto', '_num_gpus': 'auto'}
[08/01/2024 02:30:48 INFO 140444590286656] Merging with provided configuration from /opt/ml/input/config/hyperparameters.json: {'algorithm_mode': 'regular', 'feature_di

차원축소 엔드포인트 생성

In [107]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
import pandas as pd

class CustomJSONDeserializer(JSONDeserializer):
    def deserialize(self, data, content_type):
        result = super().deserialize(data, content_type)
        projections = [proj['projection'] for proj in result['projections']]
        return pd.DataFrame(projections, columns=[f'PC{i+1}' for i in range(len(projections[0]))])


pca_predictor = pca_estimator.deploy(
    initial_instance_count=1, 
    instance_type=training_instance_type,
    endpoint_name=name_from_base(project_name + '-pca'),
    serializer=CSVSerializer(),
    deserializer=CustomJSONDeserializer()
)

INFO:sagemaker:Creating model with name: adult-income-classification-2024-08-01--2024-08-01-03-11-46-170
INFO:sagemaker:Creating endpoint-config with name adult-income-classification-pca-2024-08-01-03-11-46-170
INFO:sagemaker:Creating endpoint with name adult-income-classification-pca-2024-08-01-03-11-46-170


-------!

차원축소 실행

* 훈련, 검증 데이터셋

In [135]:
# 데이터를 배치로 나누어 처리
batch_size = 10000  # 적절한 배치 크기 설정

X_train_pca = []
for i in range(0, len(X_train), batch_size):
    batch = X_train.iloc[i:i+batch_size].values
    batch_pca = pca_predictor.predict(batch)
    X_train_pca.append(batch_pca)
X_train_pca = pd.concat(X_train_pca, ignore_index=True)

X_val_pca = []
for i in range(0, len(X_val), batch_size):
    batch = X_val.iloc[i:i+batch_size].values
    batch_pca = pca_predictor.predict(batch)
    X_val_pca.append(batch_pca)
X_val_pca = pd.concat(X_val_pca, ignore_index=True)

* 테스트 데이터셋

In [123]:
# 데이터 다운로드
test_df = wr.s3.read_csv(test_path, boto3_session=boto3_session)
X_test = test_df.iloc[:, 1:]
y_test = test_df.iloc[:, 0]

In [124]:
wr.s3.download(os.path.join(asset_path, 'adult_encoders.pkl'), 'dataset/adult_encoders.pkl', boto3_session=boto3_session)
wr.s3.download(os.path.join(asset_path, 'adult_scaler.pkl'), 'dataset/adult_scaler.pkl', boto3_session=boto3_session)
with open('dataset/adult_encoders.pkl', 'rb') as f:
    feature_encoders = pickle.load(f)
with open('dataset/adult_scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [125]:
# 전처리
# '?' 값을 NaN으로 대체
X_test[X_test == '?'] = np.nan
# workclass, occupation, native-country 열은 결측치가 존재하기 때문에 이를 최빈값을 보간합니다.
X_test['workclass'].fillna(X_test['workclass'].mode()[0], inplace=True)
X_test['occupation'].fillna(X_test['occupation'].mode()[0], inplace=True)
X_test['native-country'].fillna(X_test['native-country'].mode()[0], inplace=True)
# 타겟 변수 'y'를 이진 분류를 위해 숫자로 변환
# '<=50K'와 '<=50K.'는 0으로, '>50K'와 '>50K.'는 1로 변환
y_test = y_test.replace({
    '<=50K': 0,
    '<=50K.': 0,
    '>50K': 1,
    '>50K.': 1
})

In [126]:
# 인코딩 / 스케일링
X_test_encoded = X_test.copy()
# 로드한 레이블 인코더를 사용하여 범주형 변수 인코딩
for col in feature_encoders.keys():
    X_test_encoded[col] = feature_encoders[col].transform(X_test_encoded[col])

# 수치형 변수 정규화
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test_encoded)

# 전처리된 데이터를 DataFrame으로 변환
X_test_processed = pd.DataFrame(X_test_scaled, columns=X_test_encoded.columns)

print("X_test shape:", X_test_processed.shape)
print("y_test shape:", y_test.shape)


X_test shape: (4885, 14)
y_test shape: (4885,)


In [127]:
X_test_pca = pca_predictor.predict(X_test_processed.values)
X_test_pca.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12
0,-0.400681,-0.248274,1.410914,-0.566368,0.528291,-1.953698,1.015369,-0.556811,-0.372890,1.044067,0.076347,-1.685163
1,0.243265,0.955879,-0.746418,-0.298004,0.990592,0.901442,-0.278644,-0.138798,-0.165864,1.995885,1.862311,0.306411
2,0.108381,-0.538403,-0.215819,0.504827,-0.544073,-0.636575,1.237220,0.503979,-0.613568,0.153113,-1.543314,1.950426
3,0.573499,-0.028669,1.772056,-0.052553,-0.122334,-0.554232,0.905194,0.806614,0.928657,-0.465596,-1.182949,2.758097
4,0.722200,-0.435175,0.185395,0.466918,-0.150126,-0.240334,0.872045,0.201350,-1.030158,-0.975625,0.003399,1.344921


In [136]:
pca_predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: adult-income-classification-pca-2024-08-01-03-11-46-170
INFO:sagemaker:Deleting endpoint with name: adult-income-classification-pca-2024-08-01-03-11-46-170


차원축소 데이터 S3 업로드

In [137]:
import io
import pickle
import numpy as np
from sagemaker.amazon.common import write_numpy_to_dense_tensor

# RecordIO 형식으로 데이터를 변환하고 S3에 저장하는 헬퍼 함수
def convert_to_recordio_protobuf(feature_values : np.ndarray, target_values : np.ndarray, s3_path : str):

    # RecordIO-Protobuf 형식으로 변환
    buf = io.BytesIO()
    write_numpy_to_dense_tensor(buf, feature_values, target_values)
    buf.seek(0)
    
    # S3에 업로드
    wr.s3.upload(local_file=buf, path=s3_path, boto3_session=boto3_session)
    print(f"데이터가 {s3_path}에 RecordIO-Protobuf 형식으로 저장되었습니다.")

In [149]:
convert_to_recordio_protobuf(X_train_pca.values, y_train.values.flatten(), os.path.join(pca_path, 'train/train-pca.recordio'))
convert_to_recordio_protobuf(X_val_pca.values, y_val.values.flatten(), os.path.join(pca_path, 'val/val-pca.recordio'))
convert_to_recordio_protobuf(X_test_pca.values, y_test.values.flatten(), os.path.join(pca_path, 'test/test-pca.recordio'))

데이터가 s3://dante-sagemaker/adult-income-classification/pca/train/train-pca.recordio에 RecordIO-Protobuf 형식으로 저장되었습니다.
데이터가 s3://dante-sagemaker/adult-income-classification/pca/val/val-pca.recordio에 RecordIO-Protobuf 형식으로 저장되었습니다.
데이터가 s3://dante-sagemaker/adult-income-classification/pca/test/test-pca.recordio에 RecordIO-Protobuf 형식으로 저장되었습니다.


XGBoost 빌트인 모델 및 하이퍼파라미터 설정

In [150]:
container = sagemaker.image_uris.retrieve("xgboost",sagemaker_session.boto_region_name,version="1.7-1")

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [151]:
xgb_estimator = sagemaker.estimator.Estimator(
    image_uri=container,  # 사용할 Docker 이미지의 URI
    role=role,  # IAM 역할 ARN
    sagemaker_session=sagemaker_session,  # SageMaker 세션 객체
    instance_count=1,  # 사용할 인스턴스 수
    instance_type=training_instance_type,  # 사용할 인스턴스 유형
    max_run=max_run,  # 최대 실행 시간 (초)
    use_spot_instances=use_spot_instances,  # 스팟 인스턴스 사용 여부
    max_wait=max_wait,  # 스팟 인스턴스 대기 최대 시간 (초)
    output_path=output_path,  # 모델 아티팩트 저장 경로
    base_job_name=project_name,  # 훈련 작업 이름의 기본 접두사
)

In [152]:
xgb_estimator.set_hyperparameters(
    max_depth=5,
    eta=0.1,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    objective='binary:logistic',
    num_round=200,
    early_stopping_rounds=10,
    eval_metric='logloss'
)

In [153]:
training_input_config = sagemaker.session.TrainingInput(
    s3_data=os.path.join(pca_path, 'train'),
    content_type='application/x-recordio-protobuf',
)

validation_input_config = sagemaker.session.TrainingInput(
    s3_data=os.path.join(pca_path, 'val'),
    content_type='application/x-recordio-protobuf',
)

data_channels = {'train': training_input_config, 'validation': validation_input_config}

xgb_estimator.fit(inputs=data_channels, logs=True)

INFO:sagemaker:Creating training-job with name: adult-income-classification-2024-08-01-03-32-59-239


2024-08-01 03:32:59 Starting - Starting the training job...
2024-08-01 03:33:13 Starting - Preparing the instances for training...
2024-08-01 03:33:55 Downloading - Downloading the training image......
2024-08-01 03:34:56 Training - Training image download completed. Training in progress...[2024-08-01 03:35:02.923 ip-10-0-243-133.ap-northeast-2.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-08-01 03:35:02.948 ip-10-0-243-133.ap-northeast-2.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-08-01:03:35:03:INFO] Imported framework sagemaker_xgboost_container.training
[2024-08-01:03:35:03:INFO] Failed to parse hyperparameter eval_metric value logloss to Json.
Returning the value itself
[2024-08-01:03:35:03:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2024-08-01:03:35:03:INFO] No GPUs detected (normal if no gpus installed)
[2024-08-01:03:35:03:INFO] Running XGB

In [154]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
import numpy as np

class CustomCSVDeserializer(CSVDeserializer):
    def deserialize(self, stream, content_type):
        result = super().deserialize(stream, content_type)
        return np.array([float(item[0]) for item in result])
    
    
xgb_predictor = xgb_estimator.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
    endpoint_name = name_from_base(project_name + "-xgb"),
    serialize = CSVSerializer(),
    deserializer = CustomCSVDeserializer()
)

INFO:sagemaker:Creating model with name: adult-income-classification-2024-08-01-05-04-42-748
INFO:sagemaker:Creating endpoint-config with name adult-income-classification-xgb-2024-08-01-05-04-42-748
INFO:sagemaker:Creating endpoint with name adult-income-classification-xgb-2024-08-01-05-04-42-748


-------!

In [160]:
y_preds = xgb_predictor.predict(X_test_pca)
y_preds = [1 if pred > 0.5 else 0 for pred in y_preds] # 확률값 0.5기준으로 바이너리화

In [165]:
from sklearn.metrics import classification_report

# 분류 보고서 생성
report = classification_report(y_test, y_preds)

print("분류 보고서:")
print(report)


분류 보고서:
              precision    recall  f1-score   support

           0       0.86      0.93      0.90      3699
           1       0.73      0.54      0.62      1186

    accuracy                           0.84      4885
   macro avg       0.80      0.74      0.76      4885
weighted avg       0.83      0.84      0.83      4885



In [166]:
xgb_predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: adult-income-classification-xgb-2024-08-01-05-04-42-748
INFO:sagemaker:Deleting endpoint with name: adult-income-classification-xgb-2024-08-01-05-04-42-748
